In [1]:
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
from scipy import stats 
from scipy.optimize import minimize

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

import importlib as imp
import os
import sys
sys.path.append('/home/jovyan/FR/PythonProjects/0_PDN_validation/common')

import warnings 
warnings.filterwarnings("ignore")
### модуль с полезными функциями
import useful_fun as uf
import data_validation as dv
import psi_module as pm

# Первичка

In [2]:
path = 'data/primary_08062023'
os.listdir(path)

['Описание полей первичка.xlsx',
 'eval_1.csv',
 'rosstat_mape_06_06.csv',
 'rosstat_mape_5percent_06_06.csv',
 'bki_mape_5percent_06_06.csv',
 'genpop_dev_dates_06_06.csv',
 'bki_mape_06_06.csv',
 'train_1.csv',
 'gini_06_06.csv',
 'genpop_last12months_bki_rosstat_06_06.csv']

In [3]:
df_mape_rosstat = pd.read_csv(path + '/rosstat_mape_06_06.csv', usecols = lambda x: x!="Unnamed: 0", low_memory=False)
df_mape_bki = pd.read_csv(path + '/bki_mape_06_06.csv', usecols = lambda x: x!="Unnamed: 0", low_memory=False)

In [4]:
lst_df_names = ["mape_rosstat", "mape_bki"]

In [5]:
for df_name in lst_df_names:
    exec("df_" + df_name + " = df_" + df_name + ".assign(dt = lambda x: pd.to_datetime(x.dt, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(agreementdate = lambda x: pd.to_datetime(x.agreementdate, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(loanenddate = lambda x: pd.to_datetime(x.loanenddate, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(loanenddatefact = lambda x: pd.to_datetime(x.loanenddatefact, format = '%Y-%m-%d'))")
    

In [6]:
# переименовываем для унификации
cols = {'client_pin': 'borrowerid', 'product_code': 'loantype', 'application_income': 'applincome', 'confirmed_income': 'aprincome', 'target': 'factincome',
       'model_predictions': 'modelincome', 'min_rosstat_appl': 'rosstatincome', 'bki_1_5': 'bkiincome', 'model_pdn': 'modelpdn', 
       'rosstat_appl_pdn': 'rosstatpdn', 'bki_pdn': 'bkipdn'}

for df_name in lst_df_names:
    eval("df_"+df_name).rename(columns=cols, inplace=True)

## mape_rosstat

In [7]:
df_mape_rosstat.columns

Index(['dt', 'deal_ref', 'borrowerid', 'model_id', 'segment_id', 'loantype',
       'app_position_rank', 'person_cust_gender',
       'jb_credit_limit_in_rur_mean_self', 'mean_region_income_reg', 'app_age',
       'jb_next_pay_sum_in_rur_max_active', 'jb_payments_sum_in_rur_mean_self',
       'jb_payments_sum_in_rur_max', 'jb_credit_limit_in_rur_mean_cc_self',
       'jb_next_pay_sum_in_rur_max', 'jb_credit_limit_in_rur_max_pil',
       'jb_credit_cost_rate_mean', 'jb_payments_sum_in_rur_max_cc',
       'jb_next_pay_sum_in_rur_mean_active_self',
       'jb_payments_sum_in_rur_mean_closed', 'application_income_feat', 'debt',
       'monthpayments', 'loanenddate', 'loanenddatefact', 'psk', 'regioncode',
       'applincome', 'factincome', 'aprincome', 'rosstatincome', 'bkiincome',
       'due_6_month', 'due_12_month', 'is_in_train',
       'segment_for_model_scoring', 'rosstatpdn', 'bkipdn',
       'confirmed_income_type', 'agreementdate', 'target_source',
       'maxduedays_6', 'maxdueda

In [8]:
df_mape_rosstat['fact_debt_1'] = df_mape_rosstat['modelpdn'] * df_mape_rosstat['modelincome']
df_mape_rosstat['fact_debt_2'] = df_mape_rosstat['rosstatpdn'] * df_mape_rosstat['rosstatincome']

In [9]:
df_mape_rosstat[['fact_debt_1','fact_debt_2']] # проверка: если сходится, то значит все норм 

,fact_debt_1,fact_debt_2
0,917.0,917.0
1,25342.0,25342.0
2,112646.0,112646.0
3,71279.0,71279.0
4,123030.0,123030.0
...,...,...
173038,10047.0,10047.0
173039,18688.0,18688.0
173040,241093.0,241093.0
173041,77794.0,77794.0


In [10]:
df_mape_rosstat = df_mape_rosstat.rename(columns = {'fact_debt_1':'factdebt'})

In [11]:
df_mape_rosstat['factpdn'] = df_mape_rosstat['factdebt'] / df_mape_rosstat['factincome']

In [12]:
df_mape_rosstat["ModelPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.modelpdn/0.1), 11)
df_mape_rosstat["RosstatPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.rosstatpdn/0.1), 11)
df_mape_rosstat["FactPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.factpdn/0.1),11)

# df_mape_rosstat["ModelPdnGroupName"] = np.where(df_mape_rosstat["ModelPdnGroup"] == 1, '[0;10]',
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 2, '(10;20]',
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 3, '(20;30]',        
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 4, '(30;40]',
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 5, '(40;50]', 
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 6, '(50;60]',         
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 7, '(60;70]',         
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 8, '(70;80]', 
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 9, '(80;90]', 
#                                       np.where(df_mape_rosstat["ModelPdnGroup"] == 10, '(90;100]', '100+'))))))))))

# df_mape_rosstat["RosstatPdnGroupName"] =   np.where(df_mape_rosstat["RosstatPdnGroup"] == 1, '[0;10]',
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 2, '(10;20]',
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 3, '(20;30]',        
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 4, '(30;40]',
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 5, '(40;50]', 
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 6, '(50;60]',         
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 7, '(60;70]',         
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 8, '(70;80]', 
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 9, '(80;90]', 
#                                           np.where(df_mape_rosstat["RosstatPdnGroup"] == 10, '(90;100]', '100+'))))))))))


# df_mape_rosstat["FactPdnGroupName"] =   np.where(df_mape_rosstat["FactPdnGroup"] == 1, '[0;10]',
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 2, '(10;20]',
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 3, '(20;30]',        
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 4, '(30;40]',
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 5, '(40;50]', 
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 6, '(50;60]',         
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 7, '(60;70]',         
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 8, '(70;80]', 
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 9, '(80;90]', 
#                                       np.where(df_mape_rosstat["FactPdnGroup"] == 10, '(90;100]', '100+'))))))))))

In [51]:
null_mask = (~df_mape_rosstat['modelpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['ModelPdnGroup']].value_counts(normalize=True))

In [26]:
# null_mask = (~df_mape_rosstat['modelpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
# dct = {}
# for i in range(1,12):
#     mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
#     dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['ModelPdnGroup']].value_counts(normalize=False))

In [52]:
pdn_fact_model_primary = pd.DataFrame(dct).sort_index()
pdn_fact_model_primary = pdn_fact_model_primary.fillna(0)

In [53]:
pdn_fact_model_primary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.636672,0.067150,0.005344,0.001390,0.000650,0.000371,0.000227,0.000450,0.000206,0.000000,0.000278
2.0,0.220396,0.492754,0.165314,0.034089,0.012595,0.004916,0.003745,0.004046,0.003088,0.002604,0.001851
3.0,0.058164,0.247343,0.391775,0.202999,0.083374,0.041833,0.021337,0.018729,0.013380,0.012442,0.010643
4.0,0.030908,0.085910,0.230249,0.311997,0.234438,0.142195,0.084667,0.053941,0.034788,0.033854,0.028413
5.0,0.020475,0.042593,0.097741,0.221653,0.272306,0.249977,0.186812,0.115823,0.088308,0.069444,0.049144
6.0,0.012259,0.024235,0.047348,0.103804,0.185926,0.236434,0.224038,0.198232,0.154385,0.125000,0.083758
7.0,0.007825,0.014493,0.021442,0.053036,0.099870,0.145348,0.192260,0.196284,0.183409,0.157407,0.100509
8.0,0.004564,0.008454,0.013258,0.028310,0.043475,0.077637,0.126887,0.167366,0.177851,0.164641,0.118186
9.0,0.002999,0.004670,0.009470,0.015801,0.027304,0.041833,0.067302,0.100390,0.133182,0.155093,0.116798
10.0,0.000913,0.003945,0.005547,0.009583,0.015602,0.020499,0.037680,0.059634,0.076986,0.100984,0.097455


In [29]:
# pdn_fact_model_primary.to_excel("pdnfactmodel.xlsx")

In [54]:
pdn_fact_model_primary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

In [250]:
null_mask = (~df_mape_rosstat['rosstatpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['RosstatPdnGroup']].value_counts(normalize=True))

In [56]:
pdn_fact_rosstat_primary = pd.DataFrame(dct).sort_index()
pdn_fact_rosstat_primary = pdn_fact_rosstat_primary.fillna(0)

In [57]:
pdn_fact_rosstat_primary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.636672,0.067150,0.005344,0.001390,0.000650,0.000371,0.000227,0.000450,0.000206,0.000000,0.000278
2.0,0.220396,0.492754,0.165314,0.034089,0.012595,0.004916,0.003745,0.004046,0.003088,0.002604,0.001851
3.0,0.058164,0.247343,0.391775,0.202999,0.083374,0.041833,0.021337,0.018729,0.013380,0.012442,0.010643
4.0,0.030908,0.085910,0.230249,0.311997,0.234438,0.142195,0.084667,0.053941,0.034788,0.033854,0.028413
5.0,0.020475,0.042593,0.097741,0.221653,0.272306,0.249977,0.186812,0.115823,0.088308,0.069444,0.049144
6.0,0.012259,0.024235,0.047348,0.103804,0.185926,0.236434,0.224038,0.198232,0.154385,0.125000,0.083758
7.0,0.007825,0.014493,0.021442,0.053036,0.099870,0.145348,0.192260,0.196284,0.183409,0.157407,0.100509
8.0,0.004564,0.008454,0.013258,0.028310,0.043475,0.077637,0.126887,0.167366,0.177851,0.164641,0.118186
9.0,0.002999,0.004670,0.009470,0.015801,0.027304,0.041833,0.067302,0.100390,0.133182,0.155093,0.116798
10.0,0.000913,0.003945,0.005547,0.009583,0.015602,0.020499,0.037680,0.059634,0.076986,0.100984,0.097455


In [58]:
pdn_fact_rosstat_primary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

## mape_bki

In [254]:
df_mape_bki.columns

Index(['dt', 'deal_ref', 'borrowerid', 'model_id', 'segment_id', 'loantype',
       'app_position_rank', 'person_cust_gender',
       'jb_credit_limit_in_rur_mean_self', 'mean_region_income_reg', 'app_age',
       'jb_next_pay_sum_in_rur_max_active', 'jb_payments_sum_in_rur_mean_self',
       'jb_payments_sum_in_rur_max', 'jb_credit_limit_in_rur_mean_cc_self',
       'jb_next_pay_sum_in_rur_max', 'jb_credit_limit_in_rur_max_pil',
       'jb_credit_cost_rate_mean', 'jb_payments_sum_in_rur_max_cc',
       'jb_next_pay_sum_in_rur_mean_active_self',
       'jb_payments_sum_in_rur_mean_closed', 'application_income_feat', 'debt',
       'monthpayments', 'loanenddate', 'loanenddatefact', 'psk', 'regioncode',
       'applincome', 'factincome', 'aprincome', 'rosstatincome', 'bkiincome',
       'due_6_month', 'due_12_month', 'is_in_train',
       'segment_for_model_scoring', 'rosstatpdn', 'bkipdn',
       'confirmed_income_type', 'agreementdate', 'target_source',
       'maxduedays_6', 'maxdueda

In [255]:
df_mape_bki['factdebt'] = df_mape_bki['modelpdn'] * df_mape_bki['modelincome']
df_mape_bki['factpdn'] = df_mape_bki['factdebt'] / df_mape_bki['factincome']

In [256]:
df_mape_bki["BKIPdnGroup"] = np.minimum(np.ceil(df_mape_bki.bkipdn/0.1), 11)
df_mape_bki["FactPdnGroup"] = np.minimum(np.ceil(df_mape_bki.factpdn/0.1),11)

In [257]:
null_mask = (~df_mape_bki['bkipdn'].isnull()) & (~df_mape_bki['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_bki['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_bki.loc[mask, ['BKIPdnGroup']].value_counts(normalize=True))

In [258]:
pdn_fact_bki_primary = pd.DataFrame(dct).sort_index()
pdn_fact_bki_primary = pdn_fact_bki_primary.fillna(0)

In [259]:
pdn_fact_bki_primary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.107783,0.007818,0.000927,0.000206,0.000320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2.0,0.130896,0.025606,0.005766,0.001749,0.001068,0.000986,0.000594,0.000194,0.000000,0.000000,0.000000
3.0,0.107075,0.032857,0.012459,0.006585,0.004485,0.002219,0.002523,0.001357,0.000000,0.001408,0.000744
4.0,0.084198,0.046000,0.018637,0.011832,0.007795,0.005794,0.005047,0.003102,0.003643,0.003169,0.002047
5.0,0.070519,0.048040,0.025844,0.020681,0.013454,0.010602,0.010242,0.007755,0.008847,0.004930,0.003163
6.0,0.058726,0.050986,0.032434,0.027678,0.023492,0.016889,0.013507,0.014153,0.013011,0.009155,0.008001
7.0,0.059434,0.055631,0.040980,0.034777,0.030112,0.029463,0.025976,0.023846,0.020557,0.020423,0.016560
8.0,0.055896,0.057557,0.053130,0.051960,0.049012,0.048077,0.050171,0.041295,0.042415,0.038732,0.029491
9.0,0.042689,0.058124,0.057970,0.064821,0.063962,0.062747,0.066795,0.062233,0.062191,0.041197,0.038794
10.0,0.038679,0.055858,0.060853,0.061015,0.071329,0.078895,0.070506,0.065723,0.056987,0.063732,0.051354


In [260]:
pdn_fact_bki_primary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

# Вторичка

In [30]:
path = 'data/secondary_08062023'
os.listdir(path)

['rosstat_mape_06_06.csv',
 'rosstat_mape_5percent_06_06.csv',
 'eval.csv',
 'bki_mape_5percent_06_06.csv',
 'genpop_dev_dates_06_06.csv',
 'bki_mape_06_06.csv',
 'train (2).csv',
 'gini_06_06.csv',
 'genpop_last12months_bki_rosstat_06_06.csv']

In [31]:
df_mape_rosstat = pd.read_csv(path + '/rosstat_mape_06_06.csv', usecols = lambda x: x!="Unnamed: 0", low_memory=False)
df_mape_bki = pd.read_csv(path + '/bki_mape_06_06.csv', usecols = lambda x: x!="Unnamed: 0", low_memory=False)

In [32]:
lst_df_names = ["mape_rosstat", "mape_bki"]

In [33]:
for df_name in lst_df_names:
    # exec("df_" + df_name + " = df_" + df_name + ".assign(dt = lambda x: pd.to_datetime(x.dt, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(agreementdate = lambda x: pd.to_datetime(x.agreementdate, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(loanenddate = lambda x: pd.to_datetime(x.loanenddate, format = '%Y-%m-%d'))")
    exec("df_" + df_name + " = df_" + df_name + ".assign(loanenddatefact = lambda x: pd.to_datetime(x.loanenddatefact, format = '%Y-%m-%d'))")

In [34]:
# переименовываем для унификации
cols = {'client_pin': 'borrowerid', 'product_code': 'loantype', 'application_income': 'applincome', 'confirmed_income': 'aprincome', 'target': 'factincome',
       'model_predictions': 'modelincome', 'min_rosstat_appl': 'rosstatincome', 'bki_1_5': 'bkiincome', 'model_pdn': 'modelpdn', 
       'rosstat_appl_pdn': 'rosstatpdn', 'bki_pdn': 'bkipdn'}

for df_name in lst_df_names:
    eval("df_"+df_name).rename(columns=cols, inplace=True)

## mape_rosstat

In [35]:
df_mape_rosstat.columns

Index(['deal_ref', 'deal_uk', 'borrowerid', 'agreementdate', 'model_id',
       'segment_id', 'avg_debet_turn_rur', 'mean_region_income',
       'hdb_bki_total_max_limit', 'hdb_bki_total_cc_max_limit', 'gender',
       'curr_rur_amt_cm_avg', 'avg_amount_daily_transactions_90d',
       'hdb_bki_total_pil_max_limit', 'worksalary_rur_amt',
       'avg_by_category__amount__sum__cashflowcategory_name__kafe', 'age',
       'avg_cnt_daily_transactions_90d', 'hdb_bki_active_cc_cnt',
       'hdb_bki_total_max_overdue_sum', 'transaction_category_cash_sum_cnt_m2',
       'hdb_other_active_mean_psk', 'addrref', 'debt', 'monthpayments',
       'loanenddate', 'loanenddatefact', 'regioncode', 'applincome',
       'factincome', 'target_source', 'aprincome', 'modelincome',
       'rosstatincome', 'is_in_train', 'segment_for_model_scoring',
       'rosstatpdn', 'bkipdn', 'train_segment', 'costs_source', 'modelpdn',
       'loanagreementid', 'bkiincome', 'costs', 'maxduedays_12',
       'maxduedays_6', '

In [36]:
df_mape_rosstat['factdebt'] = df_mape_rosstat['rosstatpdn'] * df_mape_rosstat['rosstatincome']
df_mape_rosstat['factpdn'] = df_mape_rosstat['factdebt'] / df_mape_rosstat['factincome']

In [37]:
df_mape_rosstat["ModelPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.modelpdn/0.1), 11)
df_mape_rosstat["RosstatPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.rosstatpdn/0.1), 11)
df_mape_rosstat["FactPdnGroup"] = np.minimum(np.ceil(df_mape_rosstat.factpdn/0.1),11)

In [44]:
null_mask = (~df_mape_rosstat['modelpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['ModelPdnGroup']].value_counts(normalize=True))

In [41]:
# null_mask = (~df_mape_rosstat['modelpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
# dct = {}
# for i in range(1,12):
#     mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
#     dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['ModelPdnGroup']].value_counts(normalize=False))

In [45]:
pdn_fact_model_secondary = pd.DataFrame(dct).sort_index()
pdn_fact_model_secondary = pdn_fact_model_secondary.fillna(0)
pdn_fact_model_secondary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.636672,0.067150,0.005344,0.001390,0.000650,0.000371,0.000227,0.000450,0.000206,0.000000,0.000278
2.0,0.220396,0.492754,0.165314,0.034089,0.012595,0.004916,0.003745,0.004046,0.003088,0.002604,0.001851
3.0,0.058164,0.247343,0.391775,0.202999,0.083374,0.041833,0.021337,0.018729,0.013380,0.012442,0.010643
4.0,0.030908,0.085910,0.230249,0.311997,0.234438,0.142195,0.084667,0.053941,0.034788,0.033854,0.028413
5.0,0.020475,0.042593,0.097741,0.221653,0.272306,0.249977,0.186812,0.115823,0.088308,0.069444,0.049144
6.0,0.012259,0.024235,0.047348,0.103804,0.185926,0.236434,0.224038,0.198232,0.154385,0.125000,0.083758
7.0,0.007825,0.014493,0.021442,0.053036,0.099870,0.145348,0.192260,0.196284,0.183409,0.157407,0.100509
8.0,0.004564,0.008454,0.013258,0.028310,0.043475,0.077637,0.126887,0.167366,0.177851,0.164641,0.118186
9.0,0.002999,0.004670,0.009470,0.015801,0.027304,0.041833,0.067302,0.100390,0.133182,0.155093,0.116798
10.0,0.000913,0.003945,0.005547,0.009583,0.015602,0.020499,0.037680,0.059634,0.076986,0.100984,0.097455


In [46]:
# pdn_fact_model_secondary.to_excel("pdnfactmodel3.xlsx")

In [40]:
pdn_fact_model_secondary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

In [272]:
null_mask = (~df_mape_rosstat['rosstatpdn'].isnull()) & (~df_mape_rosstat['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_rosstat['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_rosstat.loc[mask, ['RosstatPdnGroup']].value_counts(normalize=True))

In [273]:
pdn_fact_rosstat_secondary = pd.DataFrame(dct).sort_index()
pdn_fact_rosstat_secondary = pdn_fact_rosstat_secondary.fillna(0)
pdn_fact_rosstat_secondary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.486959,0.018921,0.001691,0.000439,0.000325,0.000278,0.000340,0.000599,0.000000,0.000000,0.000185
2.0,0.263041,0.315459,0.041193,0.011997,0.005526,0.003339,0.001816,0.003296,0.002470,0.001736,0.001481
3.0,0.075117,0.259742,0.253179,0.051865,0.017959,0.010852,0.007037,0.008840,0.005146,0.008391,0.005831
4.0,0.038472,0.137601,0.216991,0.189320,0.057858,0.019479,0.013506,0.011387,0.009675,0.007523,0.012124
5.0,0.026734,0.075282,0.150771,0.174543,0.166423,0.066135,0.022926,0.016332,0.014821,0.015336,0.016937
6.0,0.018910,0.038969,0.095712,0.134821,0.153015,0.155273,0.073885,0.022325,0.015027,0.016493,0.019806
7.0,0.013432,0.028986,0.058983,0.101463,0.123192,0.139041,0.148110,0.081061,0.022437,0.021123,0.024711
8.0,0.011868,0.018196,0.036120,0.083687,0.099464,0.113348,0.128589,0.130956,0.090984,0.025752,0.026932
9.0,0.009259,0.015539,0.023336,0.056986,0.078417,0.093127,0.107706,0.117321,0.132771,0.092303,0.029616
10.0,0.006521,0.013688,0.018128,0.038186,0.061921,0.075039,0.088526,0.103386,0.103746,0.114583,0.045627


In [274]:
pdn_fact_rosstat_secondary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

## mape_bki

In [275]:
df_mape_bki['factdebt'] = df_mape_bki['modelpdn'] * df_mape_bki['modelincome']
df_mape_bki['factpdn'] = df_mape_bki['factdebt'] / df_mape_bki['factincome']

In [276]:
df_mape_bki["BKIPdnGroup"] = np.minimum(np.ceil(df_mape_bki.bkipdn/0.1), 11)
df_mape_bki["FactPdnGroup"] = np.minimum(np.ceil(df_mape_bki.factpdn/0.1),11)

In [277]:
null_mask = (~df_mape_bki['bkipdn'].isnull()) & (~df_mape_bki['factpdn'].isnull())
dct = {}
for i in range(1,12):
    mask = (null_mask & (df_mape_bki['FactPdnGroup'] == i))
    dct['FactPdnGroup' + str(i)] = dict(df_mape_bki.loc[mask, ['BKIPdnGroup']].value_counts(normalize=True))

In [278]:
pdn_fact_bki_secondary = pd.DataFrame(dct).sort_index()
pdn_fact_bki_secondary = pdn_fact_bki_secondary.fillna(0)
pdn_fact_bki_secondary

,FactPdnGroup1,FactPdnGroup2,FactPdnGroup3,FactPdnGroup4,FactPdnGroup5,FactPdnGroup6,FactPdnGroup7,FactPdnGroup8,FactPdnGroup9,FactPdnGroup10,FactPdnGroup11
1.0,0.103282,0.002630,0.000000,0.000460,0.000096,0.000106,0.000257,0.000000,0.000231,0.000000,0.000000
2.0,0.071750,0.012714,0.003143,0.001566,0.000962,0.000531,0.000385,0.000503,0.000231,0.001316,0.000216
3.0,0.075611,0.026012,0.007502,0.003223,0.002212,0.002229,0.001539,0.001509,0.001387,0.001316,0.001401
4.0,0.065315,0.034488,0.016018,0.009210,0.004039,0.002865,0.003463,0.002850,0.003004,0.003291,0.002802
5.0,0.066281,0.037410,0.021695,0.013354,0.009424,0.005412,0.006542,0.003353,0.004622,0.002962,0.003988
6.0,0.052124,0.046032,0.035483,0.024314,0.019713,0.012310,0.009236,0.008551,0.007164,0.005923,0.006144
7.0,0.055985,0.055677,0.043187,0.036563,0.032311,0.023347,0.018343,0.010731,0.012249,0.010530,0.011102
8.0,0.055341,0.057723,0.056772,0.056732,0.043754,0.043192,0.036172,0.031690,0.023573,0.019085,0.020047
9.0,0.042471,0.060500,0.055251,0.066034,0.059621,0.061764,0.058620,0.052482,0.044604,0.035209,0.031903
10.0,0.045367,0.048224,0.057685,0.059035,0.068276,0.070678,0.073884,0.069584,0.065403,0.046068,0.044191


In [279]:
pdn_fact_bki_secondary.sum()

FactPdnGroup1     1.0
FactPdnGroup2     1.0
FactPdnGroup3     1.0
FactPdnGroup4     1.0
FactPdnGroup5     1.0
FactPdnGroup6     1.0
FactPdnGroup7     1.0
FactPdnGroup8     1.0
FactPdnGroup9     1.0
FactPdnGroup10    1.0
FactPdnGroup11    1.0
dtype: float64

## сохранение

In [280]:
def save_df(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name = sheet, startrow = 0, startcol = 0, index = False)
    writer.save()

In [281]:
dfs = [pdn_fact_model_primary, pdn_fact_rosstat_primary, pdn_fact_bki_primary, pdn_fact_model_secondary, pdn_fact_rosstat_secondary, pdn_fact_bki_secondary]
sheets = ['ПДН Модель (первичка)','ПДН Росстат (первичка)', 'ПДН БКИ (первичка)','ПДН Модель (вторичка)','ПДН Росстат (вторичка)', 'ПДН БКИ (вторичка)']

In [282]:
save_df(dfs, sheets, 'distribution_pdn.xlsx')